# Step 1: Install Python packages

In [5]:
import os
import html5lib
import pandas as pd
from selenium import webdriver                   
from selenium.webdriver.common.keys import Keys   
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options
from datetime import date, timedelta, datetime as dt
from pymongo import MongoClient
from pandas_profiling import ProfileReport




# Step 2: Preparation

In [6]:
# Connect to MongoDB
client =  MongoClient("mongodb://localhost:27017")
db = client['WallStreet']

In [7]:
class RemoteDriverStartService():
    options = webdriver.ChromeOptions()
    # Set user app data to a new directory
    options.add_argument("user-data-dir=C:\\Users\\Donley\\App Data\\Google\\Chrome\\Application\\User Data\\Kit")
    options.add_experimental_option("Proxy", "null")
    options.add_experimental_option("excludeSwitches", ["ignore-certificate-errors"])
    # Create a download path for external data sources as default: 
    options.add_experimental_option("prefs", {
      "download.default_directory": r"C:\Users\Donley\Documents\GA_TECH\SUBMISSIONS\PROJECT2-CHALLENGE\data\external",
      "download.prompt_for_download": False,
      "download.directory_upgrade": True,
      "safebrowsing.enabled": True
    }),
    # Add those optional features to capabilities
    caps = options.to_capabilities()  
    def start_driver(self):
        return webdriver.Remote(command_executor='http://127.0.0.1:4444', 
                                desired_capabilities=self.caps)
# Set class equal to new capabilities:
DesiredCapabilities = RemoteDriverStartService()  

In [8]:
# Create variables for scraping: 
wsj = "https://www.wsj.com/market-data/stocks?mod=nav_top_subsection"
# Download data to paths, csv's, json, etc: 
    # for external data sources
external = "../data/external/"
    # for processed data sources with ID's
processed = "../data/processed/"


In [16]:
current_path = os.getcwd()
Path = os.path.join(current_path, "geckodriver.exe")
# binary = FirefoxBinary(r'C:\Program Files\Mozilla Firefox\firefox.exe')
# fp = (r'C:\Users\username\AppData\Roaming\Mozilla\Firefox\Profiles\oqmqnsih.default')
opts = Options()
# opts.profile = fp
# firefox_capabilities = DesiredCapabilities.FIREFOX
# firefox_capabilities['marionette'] = True
# driver = webdriver.Firefox(capabilities=firefox_capabilities,firefox_binary=binary, firefox_options = opts)
driver = webdriver.Firefox(executable_path=Path)
driver.maximize_window()

# Initialize Chrome driver and start browser session controlled by automated test software under Kit profile.
# caps = webdriver.DesiredCapabilities.CHROME.copy()
# caps['acceptInsecureCerts'] = True
# # caps = webdriver.DesiredCapabilities.CHROME.copy()
# # caps['acceptInsecureCerts'] = True
# # driver = webdriver.Chrome(options=options, desired_capabilities=caps)
# driver = webdriver.Chrome(executable_path='chromedriver', desired_capabilities=caps)

# Get the URL
driver.get(wsj)

# Step 3: Find the IDs of the items we want to scrape for

In [17]:
# Give it time to search for ID and allow the page time to load:
timeout = 30
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, "root")))
except TimeoutException:
    driver.quit()
    

# Step 4: Techniques to make more human-like web-scrapers

In [18]:
# If the website detects us as a web-scraper, it will cut our connection so we cannot pull more data and have to re-start our scraper. This largely impacts the efficiency of the scraper and involves a lot of manual interference. There are a few techniques we can use to make the scraper more human-like:
# (1) Randomize the sleep time
# This can be easily implemented as below wherever needed:
#sleep for sometime between 5 and 8 seconds
# time.sleep(random.uniform(5,8))
# (2) Randomize the user agent for the web browser
# This is also easy and can be added to the browser options as below:
# ua = UserAgent()
# userAgent = ua.random
# Firefox_options = webdriver.FirefoxOptions()
# Firefox_options.add_argument(f’user-agent={userAgent}’)
# browser = webdriver.Firefox(executable_path = DRIVER_BIN, options=Firefox_options)
# (3) Use dynamic proxy/IP
# This requires more work than the above two. Usually free proxies are not stable and most of them don’t respond to requests, so we need to first a free proxy that responds to our requests. This website (also named as “url” in the script below) provides a lot of free proxies which we scrape down for our use. We will use Python BeautifulSoup package to scrape a list of proxies, and use Python requests package to test whether the proxy responds to our requests to the link.
# def get_proxy(link):
#     url = "https://www.sslproxies.org/"
#     r = requests.get(url)
#     soup = BeautifulSoup(r.content, 'html5lib')
#     proxies_list = list(map(lambda x: x[0]+':'+x[1], list(zip(map(lambda x: x.text, soup.findAll('td')[::8]), map(lambda x: x.text, soup.findAll('td')[1::8])))))
#     while 1:
#         try:
#             selected_ip = choice(proxies_list)
#             proxy = {'https': selected_ip, 'http': selected_ip}
#             headers = {'User-Agent': ua.random}
#             print('Using proxy:{}'.format(proxy))
#             r = requests.request('get', link, proxies=proxy, headers=headers, timeout=5)
#             break
#         except:
#             pass
        
#     return proxy
# We then add the working proxy to the browser option, similar to how we added the fake user agent:
# link = "https://www.expedia.com"
# proxy = get_proxy(link)
# Firefox_options.add_argument('--proxy-server=%s' % proxy)
# browser = webdriver.Firefox(executable_path = DRIVER_BIN, options=Firefox_options)

# Step 5: The full code that runs the scraper and save the data to .csv files

In [19]:
# Read Html to generate S&P 500 table: 
table = driver.find_element_by_id("root").get_attribute('outerHTML')
tables  = pd.read_html(table)
tables

[              Unnamed: 0      Last     Chg  %Chg
 0                   DJIA  29910.37   37.90  0.13
 1       Nasdaq Composite  12205.85  111.44  0.92
 2                S&P 500   3638.35    8.70  0.24
 3  DJ Total Stock Market  37799.36  134.63  0.36
 4           Russell 2000   1855.27   10.25  0.56
 5         NYSE Composite  14198.50    6.91  0.05
 6           Barron's 400    812.17    2.19  0.27
 7        CBOE Volatility     20.84   -0.41 -1.93
 8           DJIA Futures  29874.00   -1.00  0.00
 9        S&P 500 Futures   3640.50    4.00  0.11,
   Unnamed: 0_level_0               NYSE             Nasdaq
               Issues Unnamed: 1_level_1 Unnamed: 2_level_1
            Issues At Unnamed: 1_level_2 Unnamed: 2_level_2
         Share Volume Unnamed: 1_level_3 Unnamed: 2_level_3
 0          Advancing               1717               2251
 1          Declining               1378               1300
 2          Unchanged                101                109
 3              Total        

In [20]:
# Generate List of tables on page:
wsj_tables =[df for df in tables]

In [21]:
# Get table 5 and rename columns:
sp500_sectors_df = wsj_tables[5]
sp500_sectors_df.columns = ["S&P 500 & Sectors", "% Change"]
sp500_sectors_df.reset_index(drop=True, inplace=True)
sp500_sectors_df

,S&P 500 & Sectors,% Change
0,S&P 500,0.24
1,Communication Services,0.62
2,Consumer Discretionary,0.23
3,Consumer Staples,-0.06
4,Energy,-1.25
5,Financials,-0.42
6,Health Care,0.95
7,Industrials,-0.22
8,Information Technology,0.51
9,Materials,0.40


In [22]:
sp500_sectors_df.to_csv(external+"sp500.csv", index=False) 

In [23]:
sp500_sectors_html = sp500_sectors_df.to_html()
sp500_sectors_table = str(sp500_sectors_html)
sp500_sectors_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>S&amp;P 500 &amp; Sectors</th>\n      <th>% Change</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>S&amp;P 500</td>\n      <td>0.24</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Communication Services</td>\n      <td>0.62</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Consumer Discretionary</td>\n      <td>0.23</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Consumer Staples</td>\n      <td>-0.06</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Energy</td>\n      <td>-1.25</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Financials</td>\n      <td>-0.42</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Health Care</td>\n      <td>0.95</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>Industrials</td>\n      <td>-0.22</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>Information Technology</td>\

In [24]:
wsj_collection = db['SP500_Change']
# Insert collection
wsj_collection.update_many({},{"$set": {"S&P 500":sp500_sectors_table}})

In [25]:
# The index of the links needed
# 0	S&P 500
# 1	Communication Services
# 2	Consumer Discretionary
# 3	Consumer Staples
# 4	Energy
# 5	Financials
# 6	Health Care
# 7	Industrials
# 8	Information Technology
# 9	Materials
# 10	Real Estate
# 11	Utilities
sp500_links = [(driver.find_elements_by_partial_link_text(sector)) for sector in sp500_sectors_df["S&P 500 & Sectors"]]

# variable Links to individual sector pages: 
sp500_link = [links.get_attribute("href") for links in sp500_links[0]]
communication_link = [links.get_attribute("href") for links in sp500_links[1]]
discretionary_link = [links.get_attribute("href") for links in sp500_links[2]]
staples_link = [links.get_attribute("href") for links in sp500_links[3]]
energy_link = [links.get_attribute("href") for links in sp500_links[4]]
financials_link = [links.get_attribute("href") for links in sp500_links[5]]
health_link = [links.get_attribute("href") for links in sp500_links[6]]
industrials_link = [links.get_attribute("href") for links in sp500_links[7]]
information_Technology_link = [links.get_attribute("href") for links in sp500_links[8]]
materials_link = [links.get_attribute("href") for links in sp500_links[9]]
real_estate_link = [links.get_attribute("href") for links in sp500_links[10]]
utilities_link = [links.get_attribute("href") for links in sp500_links[11]]

In [26]:
# list of links for sectors: 
sector_links = [sp500_link, communication_link, discretionary_link, staples_link, energy_link, financials_link, health_link, industrials_link, information_Technology_link, materials_link, real_estate_link, utilities_link]
sector_dict = {"Top Sector Links": sector_links}

In [27]:
wsjlink_collection = db['SP500_Links']
wsjlink_collection.update_many({},{"$set": sector_dict} )

In [28]:
# Get to Historical Data:
driver.get(sp500_link[2])

In [29]:
historical_data = driver.find_elements(By.CSS_SELECTOR, 'a.moreLink')
historical_data

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="267a4a3a-d2d2-474e-8508-fd367d528c9d", element="733f50f6-f24b-402b-b8f6-e6e92409cdc5")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="267a4a3a-d2d2-474e-8508-fd367d528c9d", element="082a8785-78cf-47cf-8fbe-7e28873bc60e")>]

In [30]:
# Get to historical data download page: 
historical_data[1].click()

In [31]:
# Todays date
currentDate = date.today()
today = currentDate.strftime('%m/%d/%Y')
today

'11/29/2020'

In [32]:
# date 1 week ago from today
five_days = currentDate - timedelta(days=365)
five = five_days.strftime('%m/%d/%Y')
five

'11/30/2019'

In [33]:
# Fill out Date From Form
text_area = driver.find_element(By.CSS_SELECTOR, "#selectDateFrom")
text_area.send_keys(Keys.CONTROL, "a")  # or Keys.COMMAND on Mac
text_area.send_keys(five)

In [34]:
# Fill out Date to
text_area2 = driver.find_element(By.CSS_SELECTOR, "#selectDateTo")
text_area2.send_keys(Keys.CONTROL, "a")  # or Keys.COMMAND on Mac
text_area2.send_keys(today)

In [35]:
# Generate the data
generate_data = driver.find_element(By.ID, "datPickerButton")
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, "datPickerButton")))
except TimeoutException:
    driver.quit()


In [36]:
generate_data.click()

In [37]:
# Download as csv  
download_sheet = driver.find_element(By.ID, "dl_spreadsheet")
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, "dl_spreadsheet")))
except TimeoutException:
    driver.quit()

In [41]:
download_sheet.click()

In [42]:
sp_df = pd.read_csv('../data/external/HistoricalPrices.csv')

In [43]:
spx = sp_df.to_html()
sp_df_table = str(spx)
sp_df_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Date</th>\n      <th>Open</th>\n      <th>High</th>\n      <th>Low</th>\n      <th>Close</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>11/27/20</td>\n      <td>3638.55</td>\n      <td>3644.31</td>\n      <td>3629.33</td>\n      <td>3638.35</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>11/25/20</td>\n      <td>3635.50</td>\n      <td>3635.50</td>\n      <td>3617.76</td>\n      <td>3629.65</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>11/24/20</td>\n      <td>3594.52</td>\n      <td>3642.31</td>\n      <td>3594.52</td>\n      <td>3635.41</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>11/23/20</td>\n      <td>3566.82</td>\n      <td>3589.81</td>\n      <td>3552.77</td>\n      <td>3577.59</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>11/20/20</td>\n      <td>3579.31</td>\n      <td>3581.23</td>\n      <td>3556.8

In [44]:
wsj_collection = db['HP']
wsj_collection.update_one({},{"$set": {"Historical Prices":sp_df_table}})

In [45]:
prof = ProfileReport(sp_df)
prof

In [47]:
prof.to_file(output_file='../src/templates/profile.html')

In [48]:
driver.quit()